<a href="https://colab.research.google.com/github/BartBennett/SimStudent/blob/main/MM2%20general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np

In [14]:
#system state
number_in_the_system = 0

#entity attributes
class customer:
    def __init__(self, customer_ID, arrival_time, service_time, departure_time):
        self.customer_ID = customer_ID
        self.arrival_time = arrival_time
        self.service_time = service_time
        self.departure_time = departure_time

customers = []
person_served: int = -1

#resource
rho = 0 #server utilization
last_departure = 0

#events
class event:
    def __init__(self, sim_time, event_type, customer_ID):
        self.sim_time = sim_time
        self.event_type = event_type
        self.customer_ID: int = customer_ID

event_queue = []
event_queue.append(event(np.random.exponential(3.0), "Arrival", 0))

#sets
queue = []

#timing
max_time = 500
sim_time = 0

while event_queue[0].sim_time <= max_time:
    sim_time = event_queue[0].sim_time
    if event_queue[0].event_type == "Arrival":
        number_in_the_system += 1
        customers = np.append(customers, customer(event_queue[0].customer_ID, sim_time, 0, 0))
        event_queue.append(event(sim_time + np.random.exponential(3.0), "Arrival", event_queue[0].customer_ID+1))
        if number_in_the_system == 1:
            #print("Time: ", sim_time, " Arrival with immediate service: ", event_queue[0].customer_ID)
            person_served = event_queue[0].customer_ID
            customers[person_served].service_time = sim_time
            event_queue.append(event(sim_time + np.random.exponential(2.0), "Departure", event_queue[0].customer_ID))
        else:
            #print("Time: ", sim_time, " Arrival waiting in queue: ", event_queue[0].customer_ID)
            queue = np.append(queue, event_queue[0].customer_ID)
    else:   
        number_in_the_system -= 1
        customers[person_served].departure_time = sim_time
        if number_in_the_system != 0:
            #print("Time: ", sim_time, " Departure: ", person_served, " Serving: ", queue[0])
            person_served = int(queue[0])
            queue = np.delete(queue, 0)
            customers[person_served].service_time = sim_time
            event_queue.append(event(sim_time + np.random.exponential(2.0), "Departure", person_served))
        else:
            #print("Time: ", sim_time, " Departure: ", person_served, " system empty")
            person_served: int = -1
    del event_queue[0]
    event_queue.sort(key = lambda x : x.sim_time) 

last_arrival = customers[-1].customer_ID
if person_served == -1:
    person_served = last_arrival
else:
    person_served -= 1
last_departure = last_arrival   
while customers[last_departure].departure_time == 0:
    last_departure -= 1

sum_person_time = sum(customers[c].departure_time - customers[c].arrival_time for c in range(0,last_departure))
sum_person_queue_time = sum(customers[c].service_time - customers[c].arrival_time for c in range(0,person_served))
print("L = ",sum_person_time/customers[last_departure].departure_time)
print("Lq = ",sum_person_queue_time/customers[person_served].service_time)
print("W = ",sum_person_time/last_departure)
print("Wq = ",sum_person_queue_time/person_served)

L =  2.6289140861187117
Lq =  2.0131069690869365
W =  8.224644697681642
Wq =  6.293703635509626
